# Ch8. Data Wrangling: Join, Combine, and Reshape

## 8.1 Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.397038
   2   -1.864114
   3   -0.003293
b  1   -0.890991
   3    0.117094
c  1    0.613324
   2    0.441097
d  2   -0.309078
   3    0.617228
dtype: float64

In [2]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [3]:
data['b']

1   -0.890991
3    0.117094
dtype: float64

In [4]:
data['b':'c']

b  1   -0.890991
   3    0.117094
c  1    0.613324
   2    0.441097
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1   -0.890991
   3    0.117094
d  2   -0.309078
   3    0.617228
dtype: float64

In [6]:
data.loc[:,2]  # 2 is inner level selection

a   -1.864114
c    0.441097
d   -0.309078
dtype: float64

In [7]:
data.unstack()

,1,2,3
a,-0.397038,-1.864114,-0.003293
b,-0.890991,NaN,0.117094
c,0.613324,0.441097,NaN
d,NaN,-0.309078,0.617228


In [8]:
data.unstack().stack()

a  1   -0.397038
   2   -1.864114
   3   -0.003293
b  1   -0.890991
   3    0.117094
c  1    0.613324
   2    0.441097
d  2   -0.309078
   3    0.617228
dtype: float64

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                             ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [10]:
frame.index.names = ['key1', 'key2']
frame

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [11]:
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                          ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [14]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [21]:
# sort_index(level=0) or sort_index() improve the data selection performance
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [22]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [23]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame’s columns

In [24]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [26]:
frame.reset_index()

state key1 key2  Ohio     Colorado
color           Green Red    Green
0        a    1     0   1        2
1        a    2     3   4        5
2        b    1     6   7        8
3        b    2     9  10       11

In [29]:
frame.reset_index().columns

MultiIndex([(    'key1',      ''),
            (    'key2',      ''),
            (    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [35]:
frame.reset_index().set_index(('Ohio', 'Green'))

state,key1,key2,Ohio,Colorado
color,,,Red,Green
"(Ohio, Green)",,,,
0,a,1,1,2
3,a,2,4,5
6,b,1,7,8
9,b,2,10,11


## 8.2 Combining and Merging Datasets

* `pandas.merge`: `join` in SQL
* `pandas.concat`: `union` in SQL (stack)
* `combine_first`: splice data to fill in missing values

### Database-Style DataFrame Joins

In [51]:
df1 = pd.DataFrame({'key': ['a', 'b', 'c'], 'value': range(3)})
df2 = pd.DataFrame({'key': ['c', 'b', 'a', 'a'], 'value': range(4)})

In [52]:
# this merges on two keys: key and value
df1.merge(df2)

,key,value
0,b,1


In [42]:
# setting 'on' explicitly is a good practice
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,a,0,2
1,a,0,3
2,b,1,1
3,c,2,0


### Merging on Index

In [56]:
df1 = pd.DataFrame(index= ['a', 'b', 'c'], data={'value1': range(3)})
df2 = pd.DataFrame(index= ['c', 'b', 'a', 'a'], data={'value2': range(4)})

In [57]:
pd.merge(df1, df2, left_index=True, right_index=True)

,value1,value2
a,0,2
a,0,3
b,1,1
c,2,0


In [59]:
# join: merge with index. left join 
df1.join(df2)

,value1,value2
a,0,2
a,0,3
b,1,1
c,2,0


### Concatenating Along an Axis